In [61]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `~/airlab/mpc_ws/src/subcanopy_flight/notebooks/Project.toml`


In [62]:
Pkg.add("ForwardDiff")

   Resolving package versions...
  No Changes to `~/airlab/mpc_ws/src/subcanopy_flight/notebooks/Project.toml`
  No Changes to `~/airlab/mpc_ws/src/subcanopy_flight/notebooks/Manifest.toml`


In [63]:
using LinearAlgebra
using ForwardDiff
using BlockDiagonals
using ControlSystems

In [64]:
#Quaternion stuff
function hat(v)
    return [0 -v[3] v[2];
            v[3] 0 -v[1];
            -v[2] v[1] 0]
end
function L(q)
    s = q[1]
    v = q[2:4]
    L = [s    -v';
         v  s*I+hat(v)]
    return L
end
T = Diagonal([1; -ones(3)])
H = [zeros(1,3); I]
function qtoQ(q)
    return H'*T*L(q)*T*L(q)*H
end
function G(q)
    G = L(q)*H
end
function rptoq(ϕ)
    (1/sqrt(1+ϕ'*ϕ))*[1; ϕ]
end
function qtorp(q)
    q[2:4]/q[1]
end

qtorp (generic function with 1 method)

In [65]:
#Quadrotor parameters
m = 0.5
ℓ = 0.1750
J = Diagonal([0.0023, 0.0023, 0.004])
g = 9.81
kt=1.0
km=0.0245

h = 0.05 #20 Hz

Nx = 10     # number of states (quaternion)
Nx̃ = 9     # number of states (linearized)
Nu = 4     # number of controls
Tfinal = 5.0 # final time
Nt = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));

In [66]:
function E(q)
    E = BlockDiagonal([1.0*I(3), G(q), 1.0*I(3)])
end

E (generic function with 1 method)

In [67]:
# function quad_dynamics(x,u)
#     r = x[1:3]
#     q = x[4:7]/norm(x[4:7]) #normalize q just to be careful
#     v = x[8:10]
#     ω = x[11:13]
#     Q = qtoQ(q)
    
#     ṙ = Q*v
#     q̇ = 0.5*L(q)*H*ω
    
#     v̇ = Q'*[0; 0; -g] + (1/m)*[zeros(2,4); kt*ones(1,4)]*u - hat(ω)*v
    
#     ω̇ = J\(-hat(ω)*J*ω + [0 ℓ*kt 0 -ℓ*kt; -ℓ*kt 0 ℓ*kt 0; km -km km -km]*u)
    
#     return [ṙ; q̇; v̇; ω̇]
# end

# New quadrotor dynamics with control inputs as [F; wx; wy; wz]
function quad_dynamics(x,u)
    r = x[1:3]
    q = x[4:7]/norm(x[4:7]) #normalize q just to be careful
    v = x[8:10]
    ω = u[2:4]
    Q = qtoQ(q)
    
    ṙ = Q*v
    q̇ = 0.5*L(q)*H*ω
    
    v̇ = Q'*[0; 0; -g] + (1/m)*[zeros(2,4); 1 0 0 0]*u - hat(ω)*v
        
    return [ṙ; q̇; v̇]
end

quad_dynamics (generic function with 1 method)

In [68]:
function quad_dynamics_rk4(x,u)
    #RK4 integration with zero-order hold on u
    f1 = quad_dynamics(x, u)
    f2 = quad_dynamics(x + 0.5*h*f1, u)
    f3 = quad_dynamics(x + 0.5*h*f2, u)
    f4 = quad_dynamics(x + h*f3, u)
    xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    xn[4:7] .= xn[4:7]/norm(xn[4:7]) #re-normalize quaternion
    return xn
end

quad_dynamics_rk4 (generic function with 1 method)

In [69]:
#Initial Conditions
uhover = [m*g; 0; 0; 0]
r0 = [0.0; 0; 1.0]
q0 = [1.0; 0; 0; 0]
v0 = zeros(3)
x0 = [r0; q0; v0];

In [70]:
#Linearize dynamics about hover
A = ForwardDiff.jacobian(x->quad_dynamics_rk4(x,uhover),x0)
B = ForwardDiff.jacobian(u->quad_dynamics_rk4(x0,u),uhover)

10×4 Matrix{Float64}:
 0.0      0.0          0.000204375  0.0
 0.0     -0.000204375  0.0          0.0
 0.0025   0.0          0.0          0.0
 0.0      0.0          0.0          0.0
 0.0      0.025        0.0          0.0
 0.0      0.0          0.025        0.0
 0.0      0.0          0.0          0.025
 0.0      0.0          0.0122625    0.0
 0.0     -0.0122625    0.0          0.0
 0.1      0.0          0.0          0.0

In [71]:
rank(A)#The naive A matrix will always be rank deficient due to the unit-norm constraint on the quaternion

9

In [72]:
#Controlability Matrix
C = B
for k = 1:(Nx-1)
    C = [C A*C[:,end-(Nu-1):end]]
end

In [73]:
rank(C) #The naive linearized system will always be uncontrollable

9

In [74]:
@show size(A)
@show size(B)

size(A) = (10, 10)
size(B) = (10, 4)


(10, 4)

In [75]:
#Reduced system
Ã = Array(E(q0)'*A*E(q0))
B̃ = Array(E(q0)'*B);

@show size(Ã)
@show size(B̃)

size(Ã) = (9, 9)
size(B̃) = (9, 4)


(9, 4)

In [76]:
#Controlability Matrix
C = B̃
for k = 1:(Nx-1)
    C = [C Ã*C[:,end-(Nu-1):end]]
end

In [77]:
rank(C) #Reduced system is controllable

9

In [78]:
# Cost weights
Q = Array(I(Nx̃));
R = Array(.1*I(Nu));

In [79]:
#LQR Controller
K = dlqr(Ã,B̃,Q,R)

4×9 Matrix{Float64}:
 1.06627e-13   3.38709e-14   2.63944      …   1.23834e-14   3.09936
 1.38112e-13  -2.52794       1.93417e-13     -3.24277       3.08863e-14
 2.52794      -1.92617e-14   6.66105e-14     -3.93938e-14   5.04041e-14
 7.97624e-14  -4.22829e-14  -6.28036e-15     -1.37756e-14  -1.90981e-16

In [84]:
#Feedback controller
function controller(x)
    
    q0 = x0[4:7]
    q = x[4:7]
    ϕ = qtorp(L(q0)'*q)
    
    Δx̃ = [x[1:3]-r0; ϕ; x[8:10]-v0]
    
    u = uhover - K*Δx̃
end

controller (generic function with 1 method)

In [133]:
#Simulation
uhist = zeros(Nu,Nt)
xhist = zeros(Nx+3,Nt)
xhist[1:10,1] = [r0+randn(3); L(q0)*rptoq([0.2; -0.9; 0.2]); v0]
for k = 1:(Nt-1)
    uhist[:,k] = controller(xhist[1:10,k])
    xhist[1:10,k+1] = quad_dynamics_rk4(xhist[1:10,k],uhist[:,k])
    xhist[10:12,k] = uhist[2:4,k]
end

In [119]:
#Set up visualization
using TrajOptPlots
using MeshCat
using StaticArrays
using RobotZoo:Quadrotor

vis = Visualizer()
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8702


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8702))

In [125]:
model = Quadrotor()
TrajOptPlots.set_mesh!(vis, model)

false

In [135]:
X1 = [SVector{13}(x) for x in eachcol(xhist)];
visualize!(vis, model, thist[end], X1)